In [2]:
import numpy as np
from collections import Counter

# 定义节点类
class Node:
    def __init__(self, feature=None, split=None, left=None, right=None, label=None):
        self.feature = feature  # 当前节点分裂的特征索引
        self.split = split  # 分裂阈值
        self.left = left  # 左子节点
        self.right = right  # 右子节点
        self.label = label  # 如果是叶节点，存储类别
        self.is_leaf = label is not None  # 判断是否是叶节点
        

class DecisionTree:
    def __init__(self, classes, features, max_depth=10, min_samples_split=2, impurity_t='gini', n_bins=10):
        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = None
        self.n_bins = n_bins  # 分箱数量

    # 计算不纯度：支持gini和entropy
    def impurity(self, label):
        counts = np.bincount(label)
        probabilities = counts / len(label)
        if self.impurity_t == 'gini':
            return 1 - np.sum(probabilities ** 2)  # 基尼指数
        elif self.impurity_t == 'entropy':
            return -np.sum(probabilities * np.log2(probabilities + 1e-10))  # 熵，避免log0
        else:
            raise ValueError("impurity_t should be 'gini' or 'entropy'")

    def find_best_split(self, feature, label):
        best_gain = -1
        best_feature, best_split = None, None
        current_impurity = self.impurity(label)
        n_features = feature.shape[1]
        
        for i in range(n_features):
            unique_values = np.unique(feature[:, i])
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2
            
            for threshold in thresholds:
                left_indices = feature[:, i] <= threshold
                right_indices = ~left_indices
                
                if np.sum(left_indices) < 1 or np.sum(right_indices) < 1:
                    continue
                
                left_impurity = self.impurity(label[left_indices])
                right_impurity = self.impurity(label[right_indices])
                gain = current_impurity - (
                    len(label[left_indices]) / len(label) * left_impurity +
                    len(label[right_indices]) / len(label) * right_impurity
                )
                if gain > best_gain:
                    best_gain = gain
                    best_feature = i
                    best_split = threshold
        return best_feature, best_split

    # 数据分裂
    def split_data(self, feature, label, feature_idx, threshold):
        left_indices = feature[:, feature_idx] <= threshold
        right_indices = ~left_indices
        return feature[left_indices], label[left_indices], feature[right_indices], label[right_indices]

    # 创建叶节点
    def create_leaf(self, label):
        most_common_label = Counter(label).most_common(1)[0][0]
        return Node(label=most_common_label)

    def expand_node(self, feature, label, depth):
        if len(np.unique(label)) == 1 or len(label) < self.min_samples_split or depth > self.max_depth:
            return self.create_leaf(label)
        
        best_feature, best_split = self.find_best_split(feature, label)
        if best_feature is None:
            return self.create_leaf(label)
        
        left_feature, left_label, right_feature, right_label = self.split_data(feature, label, best_feature, best_split)
        left_node = self.expand_node(left_feature, left_label, depth + 1)
        right_node = self.expand_node(right_feature, right_label, depth + 1)
        return Node(feature=best_feature, split=best_split, left=left_node, right=right_node)

    def binning(self, feature):
        """
        对特征进行等宽分箱
        """
        binned_feature = np.zeros_like(feature)
        for i in range(feature.shape[1]):
            col = feature[:, i]
            bins = np.linspace(col.min(), col.max(), self.n_bins + 1)
            binned_feature[:, i] = np.digitize(col, bins[1:-1])
        # print("分箱后的特征:")
        # print(binned_feature)  # 打印分箱后的特征数据
        return binned_feature

    def fit(self, feature, label):
        assert len(self.features) == feature.shape[1], "Feature count mismatch."
        # print("训练数据特征:")
        # print(feature)  # 打印训练特征数据
        # print("训练数据标签:")
        # print(label)  # 打印训练标签数据
        # 在训练之前进行数据分箱
        binned_feature = self.binning(feature)
        self.root = self.expand_node(binned_feature, label, depth=1)

    def predict(self, feature):
        # 在预测之前也需要进行数据分箱
        binned_feature = self.binning(feature)
        if len(binned_feature.shape) == 1:
            return self.traverse_node(self.root, binned_feature)
        return np.array([self.traverse_node(self.root, sample) for sample in binned_feature])

    # 预测单个样本
    def traverse_node(self, node, sample):
        if node.is_leaf:
            return node.label
        if sample[node.feature] <= node.split:
            return self.traverse_node(node.left, sample)
        else:
            return self.traverse_node(node.right, sample)


# 测试代码
if __name__ == "__main__":
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import pandas as pd
    
    # 加载自己的数据集
    data = pd.read_csv("data.csv", sep="\t")
    
    #print(repr(data.columns))  # 打印列名的详细信息
    # 假设 'blueWins' 是标签列，表示蓝方是否获胜
    X = data.drop(columns=["gameId", "blueWins"]).values  # 删除 'gameId' 和 'blueWins' 列，作为特征
    y = data["blueWins"].values  # 'blueWins' 列是目标标签，表示蓝方是否获胜
    
    # 获取特征名称（不包括目标列）
    feature_names = data.columns.drop(["gameId", "blueWins"]).tolist()
    print("feature_names:", feature_names)
    
    # 划分训练集和测试集，80% 训练集，20% 测试集
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 定义决策树模型，包括分箱数量
    DT = DecisionTree(classes=[0, 1], features=feature_names, max_depth=2, min_samples_split=3, impurity_t='gini', n_bins=10)
    DT.fit(x_train, y_train)  # 训练模型
    p_test = DT.predict(x_test)  # 预测测试集
    
    # 输出预测结果及准确率
    print("预测结果:", p_test)
    test_acc = accuracy_score(p_test, y_test)
    print("accuracy: {:.4f}".format(test_acc))

feature_names: ['blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalGold', 'blueAvgLevel', 'blueTotalExperience', 'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff', 'blueCSPerMin', 'blueGoldPerMin', 'redWardsPlaced', 'redWardsDestroyed', 'redFirstBlood', 'redKills', 'redDeaths', 'redAssists', 'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed', 'redTotalGold', 'redAvgLevel', 'redTotalExperience', 'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff', 'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin']
预测结果: [0 1 1 ... 1 0 1]
accuracy: 0.7201
